In [1]:
%%capture
%cd ..

# Case study - LDA for the reviews of robotic vacuums

## 1. Initialisation

In [2]:
%%capture

import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()

import gc
import gensim.utils
from pyhelpers.dirs import cdd
from pyhelpers.store import save_data, load_data

from src.modeller import LatentDirichletAllocation

In [3]:
lda = LatentDirichletAllocation(product_category='vacuum', product_type='robotic')

data = lda.data.copy()
data[['review_text', 'sentiment_on_dual_scale']]

Loading "data\amazon_reviews\vacuum_cleaners\robotic\preprocd_data\preprocd_data.pkl" ... Done.


,review_text,sentiment_on_dual_scale
0,hate talk person need tech support email really,negative
1,excited receive vacuum today well package manu...,positive
2,great vacuum price work well hardwood floor gr...,positive
3,honestly want give try robot thing really work...,positive
4,hesitant buy robot simplicity robot look compl...,positive
...,...,...
77770,set run night morning wake find stick various ...,negative
77771,excellent robot vacuum come fully feature wet ...,positive
77772,good job cleaning laminate floor carpet guess ...,positive
77773,easy set use black lab pick dog hair love,positive


## 2. Preparation of input data - An example

### 2.1 Positive reviews

In [4]:
sentiment = 'positive'

docs = data[data[lda.sentiment_column_name] == sentiment][lda.review_column_name]
docs.iloc[0]

'excited receive vacuum today well package manufacturer great visual appeal surprisingly easy set install first thing notice sleek elegant design pretty impressed tire make rubber pretty solid different textured flooring'

#### 2.1.1 Get tokenized docs

In [5]:
tokenized_docs = lda.get_tokenized_docs(docs=docs, sentiment=sentiment)
tokenized_docs[0]

['excited',
 'receive',
 'package',
 'manufacturer',
 'visual',
 'appeal',
 'easy',
 'set',
 'install',
 'notice',
 'sleek',
 'elegant',
 'design',
 'pretty',
 'tire',
 'rubber',
 'pretty',
 'solid',
 'different',
 'textured',
 'flooring']

#### 2.1.2 Make a corpus

In [6]:
ngram = 3
min_count = 1
threshold = 10e-5

corpus, id2word, texts = lda.make_corpus(
    tokenized_docs=tokenized_docs, ngram=ngram, min_count=min_count, threshold=threshold, 
    scoring='npmi')

In [7]:
# `texts` is like a 3-gram version of `tokenized_docs`
texts[0]

['excited_receive',
 'package_manufacturer',
 'visual_appeal',
 'easy_set',
 'install_notice',
 'sleek_elegant',
 'design_pretty',
 'tire_rubber',
 'pretty_solid',
 'different_textured',
 'flooring']

In [8]:
# `corpus` is like an encoded version of `texts`
corpus[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1)]

### 2.2 Negative reviews

In [9]:
sentiment = 'negative'

docs = data[data[lda.sentiment_column_name] == sentiment][lda.review_column_name]
docs.iloc[0]

'hate talk person need tech support email really'

#### 2.2.1 Get tokenized docs

In [10]:
tokenized_docs = lda.get_tokenized_docs(docs=docs, sentiment=sentiment)
tokenized_docs[0]

['hate', 'talk', 'person', 'tech', 'support', 'email']

#### 2.2.2 Make a corpus

In [11]:
ngram = 3
min_count = 1
threshold = 10e-5

corpus, id2word, texts = lda.make_corpus(
    tokenized_docs=tokenized_docs, ngram=ngram, min_count=min_count, threshold=threshold, 
    scoring='npmi')

In [12]:
texts[0]

['hate_talk', 'person_tech', 'support_email']

In [13]:
corpus[0]

[(0, 1), (1, 1), (2, 1)]

## 3. Evaluation of LDA models on the review texts

In [14]:
# lda.evaluate_models()  # (with different sets of hyper-parameters)

### 3.1 Positive reviews

In [15]:
pos_lda_eval_summary = lda.fetch_evaluation_summary(sentiment='positive').query('alpha != "auto"')
pos_lda_eval_summary.head()

,min_count,threshold,corpus_proportion,num_topics,alpha,eta,coherence_score
76,5,0.0001,90%,3,asymmetric,1.0,0.598883
98,1,0.0001,80%,3,asymmetric,symmetric,0.594837
123,1,0.0001,85%,3,asymmetric,auto,0.588380
129,5,0.0001,95%,3,asymmetric,1.0,0.587385
136,1,0.0001,90%,3,asymmetric,symmetric,0.586332


In [16]:
pos_idx = 76
pos_key = f'LDA_{str(pos_idx).zfill(3)}'

# pos_lda_vis_data = lda.get_vis_data(sentiment='positive', i=pos_idx, export_to_html=True, update=True, verbose=True)
pos_lda_vis_data = lda.get_vis_data(sentiment='positive', i=pos_idx)

In [17]:
pos_lda_vis_data[pos_key]

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.086282  0.030814       1        1  64.487023
1     -0.015288 -0.049534       2        1  31.064750
2      0.101570  0.018720       3        1   4.448227, topic_info=       Term         Freq        Total Category  logprob  loglift
2623    mop  1583.000000  1583.000000  Default  30.0000  30.0000
2485    pad   439.000000   439.000000  Default  29.0000  29.0000
4492  water   396.000000   396.000000  Default  28.0000  28.0000
1672   unit  1395.000000  1395.000000  Default  27.0000  27.0000
77     room  2261.000000  2261.000000  Default  26.0000  26.0000
...     ...          ...          ...      ...      ...      ...
237     try    44.622289  1506.617787   Topic3  -7.0270  -0.4067
474     set    42.497211  1623.974959   Topic3  -7.0758  -0.5305
77     room    42.132213  2261.194446   Topic3  -7.0845  -0.8702
333    time    41.170964  3862.836905   Topic3  -7.1075  -1.4288
342    area    35.293301  2021.971459   Topic3  -7.2616  -0.9355

[285 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
1421       1  0.466668        add
1421       2  0.474165        add
1421       3  0.059973        add
22145      1  0.055749   add_room
22145      2  0.891977   add_room
...      ...       ...        ...
1233       2  0.022000  wonderful
1233       3  0.004000  wonderful
5074       1  0.161091       zone
5074       2  0.835129       zone
5074       3  0.004239       zone

[698 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])

### 3.2 Negative reviews

In [18]:
neg_eval_result_summary = lda.fetch_evaluation_summary(sentiment='negative')
neg_eval_result_summary.head()

,min_count,threshold,corpus_proportion,num_topics,alpha,eta,coherence_score
0,5,0.0001,85%,3,auto,1.0,0.667406
1,1,0.0001,90%,10,auto,symmetric,0.626780
2,1,0.0001,90%,10,auto,auto,0.626780
3,1,0.0001,85%,10,auto,auto,0.622343
4,1,0.0001,85%,10,auto,symmetric,0.622343


In [19]:
neg_idx = 0
neg_key = f'LDA_{str(neg_idx).zfill(3)}'

# neg_lda_vis_data = lda.get_vis_data(sentiment='negative', i=neg_idx, export_to_html=True, update=True, verbose=True)
neg_lda_vis_data = lda.get_vis_data(sentiment='negative', i=neg_idx)

In [20]:
neg_lda_vis_data[neg_key]

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.115395 -0.065459       1        1   0.421455
1     -0.128202 -0.047623       2        1  70.374010
2      0.012807  0.113082       3        1  29.204535, topic_info=                 Term        Freq        Total Category  logprob  loglift
38   customer_service  456.000000   456.000000  Default  30.0000  30.0000
761             month  604.000000   604.000000  Default  29.0000  29.0000
27        replacement  283.000000   283.000000  Default  28.0000  28.0000
36            company  235.000000   235.000000  Default  27.0000  27.0000
802              call  252.000000   252.000000  Default  26.0000  26.0000
..                ...         ...          ...      ...      ...      ...
31               time  195.009735  1011.392332   Topic3  -5.8142  -0.4152
395             still  160.209717   447.821926   Topic3  -6.0108   0.2029
192            return  172.596099   816.618139   Topic3  -5.9363  -0.3234
406           machine  159.217571   451.382714   Topic3  -6.0170   0.1888
34               back  160.666468   614.265126   Topic3  -6.0079  -0.1102

[207 rows x 6 columns], token_table=       Topic      Freq           Term
term                                 
9783       1  0.052832        abandon
9783       2  0.528322        abandon
9783       3  0.475490        abandon
4032       1  0.039736         action
4032       2  0.715243         action
...      ...       ...            ...
954        2  0.018499  year_warranty
954        3  0.980431  year_warranty
15787      1  0.155868   yet_continue
15787      2  0.779340   yet_continue
15787      3  0.155868   yet_continue

[377 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])

---